# Assignment 10: Modeling Fundamentals

Complete the following three questions to demonstrate your understanding of statistical modeling, machine learning, and gradient boosting.

## Setup

In [55]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
import xgboost as xgb
import os

# Create output directory
os.makedirs("output", exist_ok=True)

# Set random seed for reproducibility
np.random.seed(42)

## Load Data

In [56]:
# Load California Housing dataset from scikit-learn
from sklearn.datasets import fetch_california_housing

# Fetch the dataset
housing_data = fetch_california_housing(as_frame=True)
df = housing_data.frame

# Rename target for clarity
df = df.rename(columns={"MedHouseVal": "house_value"})

print(f"Loaded {len(df)} housing records")
print("\nFeature names:", housing_data.feature_names)
print("\nFirst few rows:")
print(df.head())
print("\nSummary statistics:")
print(df.describe())

Loaded 20640 housing records

Feature names: ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']

First few rows:
   MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  Latitude  \
0  8.3252      41.0  6.984127   1.023810       322.0  2.555556     37.88   
1  8.3014      21.0  6.238137   0.971880      2401.0  2.109842     37.86   
2  7.2574      52.0  8.288136   1.073446       496.0  2.802260     37.85   
3  5.6431      52.0  5.817352   1.073059       558.0  2.547945     37.85   
4  3.8462      52.0  6.281853   1.081081       565.0  2.181467     37.85   

   Longitude  house_value  
0    -122.23        4.526  
1    -122.22        3.585  
2    -122.24        3.521  
3    -122.25        3.413  
4    -122.25        3.422  

Summary statistics:
             MedInc      HouseAge      AveRooms     AveBedrms    Population  \
count  20640.000000  20640.000000  20640.000000  20640.000000  20640.000000   
mean       3.870671     28.639486  

---

## Question 1: Statistical Modeling with statsmodels

**Note:** This question focuses on statistical modeling for inference - understanding relationships between variables. We'll use a subset of features (`MedInc`, `AveBedrms`, `Population`) to focus on interpretability and statistical significance rather than maximizing prediction accuracy. The `statsmodels` library provides detailed statistical information (p-values, confidence intervals, AIC) that helps us understand *why* variables are related.

**Why a subset of features?** In statistical modeling, we often use fewer features to maintain interpretability and focus on understanding relationships. This contrasts with machine learning (Question 2), where we use all available features to maximize prediction accuracy.

**Objective:** Fit linear regression models using `statsmodels`, extract statistical information, and compare models with and without interaction terms.

### Part 1.1: Fit the Model

Fit a linear regression model predicting `house_value` from `MedInc`, `AveBedrms`, and `Population` using the formula API.

In [57]:
# Fit a linear regression model using statsmodels formula API
model = smf.ols('house_value ~ MedInc + AveBedrms + Population', data = df)  
results = model.fit()  

### Part 1.2: Extract Model Summary

Print the model summary and save key statistics to a text file.

In [58]:
# Print the model summary
print("=== Model Summary ===")
print(results.summary())

# Extract p-values for coefficients
# Print which coefficients are statistically significant (p < 0.05)

pvalues = results.pvalues 
print("\n=== Coefficient Significance (p-values) ===\n")
print("=== All p-values ===\n")
print(pvalues)
print("\n=== Statistically significant (<0.05) p-values ===\n")
for coefficient, pvalue in pvalues.items():
    if pvalue < 0.05:
        print(f"Statistically significant coefficient: {coefficient} || p-value: {pvalue:.2f}\n")

# Save key statistics to output file
# Extract: R-squared, number of observations, and AIC (Akaike Information Criterion)
# Format: "R-squared: X.XXXX\nObservations: XXXX\nAIC: XXXXX.XX"

with open("output/q1_model_summary.txt", "w") as f:
    f.write(f"R-squared: {results.rsquared:.4f}\n")
    f.write(f"Observations: {int(results.nobs)}\n")
    f.write(f"AIC: {results.aic:.2f}")

=== Model Summary ===
                            OLS Regression Results                            
Dep. Variable:            house_value   R-squared:                       0.474
Model:                            OLS   Adj. R-squared:                  0.474
Method:                 Least Squares   F-statistic:                     6205.
Date:                Sat, 29 Nov 2025   Prob (F-statistic):               0.00
Time:                        15:04:35   Log-Likelihood:                -25607.
No. Observations:               20640   AIC:                         5.122e+04
Df Residuals:                   20636   BIC:                         5.125e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.5084      0.0

### Part 1.3: Make Predictions

Make predictions for all houses and save to CSV.

**Note:** In statistical modeling, we often make predictions on the full dataset to understand model fit. This differs from machine learning (Question 2), where we use train/test splits to evaluate generalization.

In [59]:
# Make predictions using the fitted model
# The features are: MedInc, AveBedrms, Population
# Save predictions along with actual values to CSV
predictions = results.predict()  

# Create DataFrame with predictions
pred_df = pd.DataFrame({
    "actual_value": df["house_value"],
    "predicted_value": predictions,
})

# Save to CSV
pred_df.to_csv("output/q1_statistical_model.csv", index=False)
print(f"\nSaved {len(pred_df)} predictions to output/q1_statistical_model.csv")


Saved 20640 predictions to output/q1_statistical_model.csv


### Part 1.4: Model with Interaction Term

Now let's fit a model with an interaction term. An **interaction term** allows the effect of one variable to depend on the value of another variable. For example, the effect of income (`MedInc`) on house value might depend on the number of bedrooms (`AveBedrms`). In the formula API, we use `*` to include both main effects and their interaction.

In [60]:
# Fit a model with an interaction term between MedInc and AveBedrms
model_interaction = smf.ols('house_value ~ MedInc * AveBedrms + Population', data = df) 
results_interaction = model_interaction.fit()  
print("\n=== Model with Interaction Term ===")
print(results_interaction.summary())


=== Model with Interaction Term ===
                            OLS Regression Results                            
Dep. Variable:            house_value   R-squared:                       0.474
Model:                            OLS   Adj. R-squared:                  0.474
Method:                 Least Squares   F-statistic:                     4655.
Date:                Sat, 29 Nov 2025   Prob (F-statistic):               0.00
Time:                        15:04:36   Log-Likelihood:                -25605.
No. Observations:               20640   AIC:                         5.122e+04
Df Residuals:                   20635   BIC:                         5.126e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Int

### Part 1.5: Compare Models

Compare the two models using AIC (Akaike Information Criterion). Lower AIC indicates a better model (accounting for model complexity).

In [61]:
# Compare the two models using AIC
# Extract AIC from both models: results.aic and results_interaction.aic
# Determine which model is better (lower AIC is better)

aic_simple = results.aic  
aic_interaction = results_interaction.aic  

print("\n=== Model Comparison ===")
print(f"Simple model AIC: {aic_simple:.2f}")
print(f"Interaction model AIC: {aic_interaction:.2f}")
if aic_simple > aic_interaction:
    print(f"The interaction model is better, as it has a lower AIC: {aic_interaction:.2f}")
elif aic_simple < aic_interaction:
    print(f"The simple model is better, as it has a lower AIC: {aic_simple:.2f}")
else:
    print(f"Both models perform the same, as they both have an AIC of: {aic_simple:.2f}")




=== Model Comparison ===
Simple model AIC: 51221.28
Interaction model AIC: 51220.38
The interaction model is better, as it has a lower AIC: 51220.38


---

## Question 2: Machine Learning with scikit-learn

**Note:** While Question 1 focused on statistical inference (understanding relationships and testing hypotheses), Question 2 focuses on machine learning for prediction. We'll use all available features to maximize prediction accuracy rather than focusing on interpretability.

**Objective:** Fit and compare linear regression and random forest models using `scikit-learn`.

### Part 2.1: Prepare Data

Split the data into training and test sets.

In [62]:
# Prepare features and target
# Features: ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
# Target: 'house_value'

feature_cols = [
    "MedInc",
    "HouseAge",
    "AveRooms",
    "AveBedrms",
    "Population",
    "AveOccup",
    "Latitude",
    "Longitude",
]

X = df[feature_cols]  
y = df["house_value"] 

# Split into train and test sets (80/20 split, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42) 

print(f"Training set: {len(X_train)} samples")
print(f"Test set: {len(X_test)} samples")

Training set: 16512 samples
Test set: 4128 samples


### Part 2.2: Fit Linear Regression

Fit a linear regression model and evaluate it on both training and test sets. Comparing train and test performance helps us detect overfitting - if the model performs much better on training data than test data, it's likely overfitting.

In [63]:
# Fit a LinearRegression model
lr_model = LinearRegression()  
lr_model.fit(X_train, y_train)


# Make predictions on both training and test sets
lr_train_pred = lr_model.predict(X_train) 
lr_test_pred = lr_model.predict(X_test)  

# Calculate metrics on both sets
lr_train_r2 = r2_score(y_train, lr_train_pred)
lr_test_r2 = r2_score(y_test, lr_test_pred)
lr_train_rmse = np.sqrt(mean_squared_error(y_train, lr_train_pred))
lr_test_rmse = np.sqrt(mean_squared_error(y_test, lr_test_pred))

print("=== Linear Regression Results ===")
print(f"Training - R²: {lr_train_r2:.4f}, RMSE: {lr_train_rmse:.2f}")
print(f"Test - R²: {lr_test_r2:.4f}, RMSE: {lr_test_rmse:.2f}")

# Store test predictions for later use
lr_pred = lr_test_pred
lr_r2 = lr_test_r2
lr_rmse = lr_test_rmse

=== Linear Regression Results ===
Training - R²: 0.6126, RMSE: 0.72
Test - R²: 0.5758, RMSE: 0.75


### Part 2.3: Fit Random Forest

Fit a random forest model and evaluate it on both training and test sets.

In [64]:
# Fit a RandomForestRegressor model
# Use: n_estimators=50, max_depth=8, random_state=42
rf_model = RandomForestRegressor(n_estimators=50, max_depth=8, random_state=42)  
rf_model.fit(X_train, y_train)

# Make predictions on both training and test sets
rf_train_pred = rf_model.predict(X_train) 
rf_test_pred = rf_model.predict(X_test) 

# Calculate metrics on both sets
rf_train_r2 = r2_score(y_train, rf_train_pred)
rf_test_r2 = r2_score(y_test, rf_test_pred)
rf_train_rmse = np.sqrt(mean_squared_error(y_train, rf_train_pred))
rf_test_rmse = np.sqrt(mean_squared_error(y_test, rf_test_pred))

print("=== Random Forest Results ===")
print(f"Training - R²: {rf_train_r2:.4f}, RMSE: {rf_train_rmse:.2f}")
print(f"Test - R²: {rf_test_r2:.4f}, RMSE: {rf_test_rmse:.2f}")

# Store test predictions and metrics for later use
rf_pred = rf_test_pred
rf_r2 = rf_test_r2
rf_rmse = rf_test_rmse

# Extract feature importance for later comparison
rf_feature_importance = rf_model.feature_importances_ 

=== Random Forest Results ===
Training - R²: 0.8050, RMSE: 0.51
Test - R²: 0.7389, RMSE: 0.58


### Part 2.4: Save Predictions and Comparison

Save predictions and model comparison to files.

In [65]:
# Save predictions to CSV
# Include: actual_value, lr_predicted_value, rf_predicted_value
pred_df = pd.DataFrame({
    "actual_value": y_test.values,
    "lr_predicted_value": lr_pred,
    "rf_predicted_value": rf_pred,
})

pred_df.to_csv("output/q2_ml_predictions.csv", index=False)
print(f"\nSaved predictions to output/q2_ml_predictions.csv")

# Save model comparison to text file
# Include both train and test metrics
# Format: "Linear Regression - Train R²: X.XXXX, Test R²: X.XXXX, Test RMSE: XX.XX\nRandom Forest - Train R²: X.XXXX, Test R²: X.XXXX, Test RMSE: XX.XX"
with open("output/q2_model_comparison.txt", "w") as f:
    f.write(f"Linear Regression - Train R²: {lr_train_r2:.4f}, Test R²: {lr_r2:.4f}, Test RMSE: {lr_rmse:.2f}\n")
    f.write(f"Random Forest - Train R²: {rf_train_r2:.4f}, Test R²: {rf_r2:.4f}, Test RMSE: {rf_rmse:.2f}")


Saved predictions to output/q2_ml_predictions.csv


---

## Question 3: Gradient Boosting with XGBoost

**Note:** Question 3 introduces gradient boosting, an advanced machine learning technique that often achieves the best performance on tabular data. XGBoost builds models sequentially, with each new model learning from the mistakes of previous ones.

**Objective:** Fit an XGBoost model and extract feature importance.

### Part 3.1: Fit XGBoost Model

Fit an XGBoost regressor model.

In [66]:
# Fit an XGBRegressor model
# Use: n_estimators=100, max_depth=3, learning_rate=0.15, random_state=42
xgb_model = xgb.XGBRegressor(n_estimators=100, max_depth=3, learning_rate=0.15, random_state=42)  # Replace None with your code
xgb_model.fit(X_train, y_train)

# Make predictions on test set
xgb_pred = xgb_model.predict(X_test)  # Replace None with your code

# Calculate metrics
xgb_r2 = r2_score(y_test, xgb_pred)
xgb_rmse = np.sqrt(mean_squared_error(y_test, xgb_pred))

print(f"XGBoost - R²: {xgb_r2:.4f}, RMSE: {xgb_rmse:.2f}")

XGBoost - R²: 0.7918, RMSE: 0.52


### Part 3.2: Extract and Compare Feature Importance

Extract feature importance from XGBoost and compare it with Random Forest from Question 2.

In [67]:
# Extract feature importance from XGBoost
# Use: xgb_model.feature_importances_
xgb_feature_importance = xgb_model.feature_importances_  

# Create DataFrame for XGBoost importance
xgb_importance_df = pd.DataFrame({
    "feature": feature_cols,
    "xgb_importance": xgb_feature_importance,
}).sort_values("xgb_importance", ascending=False)

print("\n=== XGBoost Feature Importance ===")
print(xgb_importance_df)

# Compare with Random Forest feature importance
# Create a comparison DataFrame with both models' feature importance
# Sort by XGBoost importance for display
importance_comparison = pd.DataFrame({
    "feature": feature_cols,
    "random_forest": rf_feature_importance,
    "xgboost": xgb_feature_importance,
}).sort_values("xgboost", ascending=False)

print("\n=== Feature Importance Comparison ===")
print(importance_comparison)

# Save XGBoost feature importance to text file
# Format: "feature_name: X.XXXX" (one per line, sorted by importance)
with open("output/q3_feature_importance.txt", "w") as f:
    for feature, importance in zip(xgb_importance_df['feature'], xgb_importance_df['xgb_importance']):
        f.write(f"{feature}: {importance:.4f}\n")



=== XGBoost Feature Importance ===
      feature  xgb_importance
0      MedInc        0.567666
5    AveOccup        0.154247
7   Longitude        0.074295
1    HouseAge        0.067540
6    Latitude        0.057086
2    AveRooms        0.053436
3   AveBedrms        0.014782
4  Population        0.010947

=== Feature Importance Comparison ===
      feature  random_forest   xgboost
0      MedInc       0.644830  0.567666
5    AveOccup       0.140574  0.154247
7   Longitude       0.060245  0.074295
1    HouseAge       0.046593  0.067540
6    Latitude       0.062580  0.057086
2    AveRooms       0.025040  0.053436
3   AveBedrms       0.009721  0.014782
4  Population       0.010418  0.010947


### Part 3.3: Save Predictions

Save XGBoost predictions to CSV.

In [68]:
# Save predictions to CSV
# Include: actual_value, xgb_predicted_value
pred_df = pd.DataFrame({
    "actual_value": y_test.values,
    "xgb_predicted_value": xgb_pred,
})

pred_df.to_csv("output/q3_xgboost_model.csv", index=False)
print(f"\nSaved predictions to output/q3_xgboost_model.csv")


Saved predictions to output/q3_xgboost_model.csv


---

## Submission Checklist

Before submitting, verify you've created all required output files:

- [ ] `output/q1_statistical_model.csv`
- [ ] `output/q1_model_summary.txt`
- [ ] `output/q2_ml_predictions.csv`
- [ ] `output/q2_model_comparison.txt`
- [ ] `output/q3_xgboost_model.csv`
- [ ] `output/q3_feature_importance.txt`
